In [3]:
import pandas as pd # dataframes and csv
import numpy as np # linear algebra
import sklearn as sk # data maniputation
import matplotlib.pyplot as plt # data visualization
import re # regular expressions for the text
import tensorflow as tf # for ML


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tokenizers import SentencePieceBPETokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import PreTokenizer
from tokenizers import SentencePieceBPETokenizer


In [5]:
reviews_train = pd.read_csv("./archive/train.csv",header=None, names=['sentiment', 'short_text','text'])
reviews_test = pd.read_csv("./archive/test.csv",header=None, names=['sentiment', 'short_text','text'])

In [6]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Loop over all training examples and save the sentences and labels
for t in reviews_train["text"]:
  training_sentences.append(t)

for t in reviews_train["short_text"]:
  training_sentences.append(t)


for s in reviews_train["sentiment"]:
  if s == 2: # Redo the labels so the processing is faster 
    training_labels.append(1)
  else:
    training_labels.append(0)
#do it for the second time so short-text can have sentiments
for s in reviews_train["sentiment"]:
  if s == 2: 
    training_labels.append(1)
  else:
    training_labels.append(0)

# Loop over all test examples and save the sentences and labels
for t in reviews_test["text"]:
  testing_sentences.append(t)

for t in reviews_test["short_text"]:
  testing_sentences.append(t)


for s in reviews_test["sentiment"]:
  if s == 2: # Redo the labels so the processing is faster 
    testing_labels.append(1)
  else:
    testing_labels.append(0)
#do it for the second time so short-text can have sentiments
for s in reviews_test["sentiment"]:
  if s == 2: 
    testing_labels.append(1)
  else:
    testing_labels.append(0)

# Convert labels lists to numpy array
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


print(len(training_labels_final))

7200000


In [7]:
print(sum(training_labels_final))

3600000


In [59]:
sentences = []
max_len=0
pattern = r'[,\s!?()._\-]+'
for sentence in training_sentences:
    sentence = str(sentence).lower()
    sentence = re.split(pattern, sentence) # Split the text based on the pattern
    if len(sentence)>max_len:
        max_len=len(sentence)
    sentence = " ".join(sentence)
    sentences.append(sentence)
    
print(max_len)

254


In [ ]:
with open("text_2.txt", "w", encoding="utf-8") as f:
    for sentence in sentences:
        f.write(sentence+"\n")
    f.close()

In [9]:
vocab_size = 10000
trunc_type='pre'
oov_tok = "<OOV>"

BPE_sentence_tokenizer = SentencePieceBPETokenizer()
BPE_sentence_tokenizer.train("text_2.txt",vocab_size=vocab_size)

encoded_texts = [BPE_sentence_tokenizer.encode(text).ids for text in sentences]
# Pad the sequences to a consistent length
X = pad_sequences(encoded_texts, maxlen=300, padding='pre')
print(X)
y = training_labels_final


[[   0    0    0 ...   51  123   51]
 [   0    0    0 ...  455 5124  123]
 [   0    0    0 ...  978  981  123]
 ...
 [   0    0    0 ...  663  156 1223]
 [   0    0    0 ...  163 2313  123]
 [   0    0    0 ... 1768  158 2220]]


In [65]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=BPE_sentence_tokenizer)
BPE_sentence_tokenizer.save('./my_additionally_trained_tokenizer_BPE.json')

In [12]:
# Initialize or load your NLP model
model = tf.keras.models.load_model("./my_trained_model")

In [16]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
optimizer = tf.keras.optimizers.Adam(learning_rate=0.007)
X_train, X_val, y_train, y_val = train_test_split(X[:100000], y[:100000], test_size=0.3, random_state=42)


In [62]:
print(BPE_sentence_tokenizer.get_vocab_size())

10000


In [ ]:
model.fit(X_train,y_train,batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=(X_val, y_val))

Epoch 1/5
547/547 [==============================] - 141s 247ms/step - loss: 0.4900 - accuracy: 0.7544 - val_loss: 0.3554 - val_accuracy: 0.8498
Epoch 2/5
547/547 [==============================] - 178s 325ms/step - loss: 0.3089 - accuracy: 0.8742 - val_loss: 0.3048 - val_accuracy: 0.8750
Epoch 3/5
547/547 [==============================] - 186s 340ms/step - loss: 0.2508 - accuracy: 0.9017 - val_loss: 0.2982 - val_accuracy: 0.8824
Epoch 4/5
547/547 [==============================] - 194s 355ms/step - loss: 0.2091 - accuracy: 0.9203 - val_loss: 0.2815 - val_accuracy: 0.8870
Epoch 5/5
547/547 [==============================] - 186s 341ms/step - loss: 0.1743 - accuracy: 0.9364 - val_loss: 0.2905 - val_accuracy: 0.8818


In [ ]:
# Assuming you have trained your model and stored it in the `model_lstm` variable
# Specify the path where you want to save the model
model_save_path = './my_additionally_trained_model'

# Save the model
model.save(model_save_path)

NameError: name 'model' is not defined

In [13]:
# Initialize or load your NLP model
model = tf.keras.models.load_model('./my_additionally_trained_model')

In [57]:
#THE INPUT SHOULD BE ALWAYS LOWER
test = 0
for i in range(-1,-50,-1):
    input_sentence = testing_sentences[i].lower()
    # Preprocess the sentence (lowercase, tokenize, and pad)
    input_sequence = BPE_sentence_tokenizer.encode(input_sentence).ids
    input_sequence = pad_sequences([input_sequence], maxlen=100, padding='pre')
    # Make a prediction
    predicted_probability = model.predict(input_sequence)

    # Interpret the prediction (assuming binary classification)
    if predicted_probability > 0.5:
        sentiment = 1
        if sentiment==testing_labels[i]:
            test=test+1
        else:
            print(f"sentence:{input_sentence} right answer{testing_labels[i]} my answer {sentiment} (Probability: {predicted_probability[0][0]}")
    else:
        sentiment = 0
        if sentiment==testing_labels[i]:
            test=test+1
        else:
            print(f"sentence:{input_sentence} right answer{testing_labels[i]} my answer {sentiment} (Probability: {predicted_probability[0][0]}")


1/1 [==============================] - 0s 223ms/step
sentence:comedy scene, and not heard right answer0 my answer 1 (Probability: 0.573357343673706
1/1 [==============================] - 0s 48ms/step
sentence:strange but, listenable right answer0 my answer 1 (Probability: 0.6397700309753418
1/1 [==============================] - 0s 50ms/step
sentence:super. right answer1 my answer 0 (Probability: 0.4396395981311798
1/1 [==============================] - 0s 29ms/step
sentence:mac os x server administrator's guide right answer0 my answer 1 (Probability: 0.8991463780403137
1/1 [==============================] - 0s 32ms/step
sentence:huh? the cd's over? best nap i ever took. right answer0 my answer 1 (Probability: 0.9019368290901184
1/1 [==============================] - 0s 45ms/step
sentence:neil armstrong, a good man right answer0 my answer 1 (Probability: 0.9510635733604431
1/1 [==============================] - 0s 38ms/step
sentence:okay at first, but died pretty quickly... right answe

In [58]:
print(test)

36
